In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8c57fbc8",
   "metadata": {},
   "outputs": [],
   "source": [
    "import gradio as gr\n",
    "import os\n",
    "import json\n",
    "import numpy as np\n",
    "from dotenv import load_dotenv\n",
    "from langchain_groq import ChatGroq\n",
    "from sentence_transformers import SentenceTransformer\n",
    "import faiss\n",
    "load_dotenv()\n",
    "GROQ_API_KEY = os.getenv(\"GROQ_API_KEY\")\n",
    "llm = ChatGroq(model=\"llama-3.1-8b-instant\")\n",
    "embedding_model = SentenceTransformer(\"all-MiniLM-L6-v2\")\n",
    "d = 384\n",
    "if os.path.exists(\"user_history.index\"):\n",
    "    index = faiss.read_index(\"user_history.index\")\n",
    "else:\n",
    "    index = faiss.IndexFlatL2(d)\n",
    "def load_user_history():\n",
    "    if os.path.exists(\"user_history.json\"):\n",
    "        with open(\"user_history.json\", \"r\") as file:\n",
    "            return json.load(file)\n",
    "    return []\n",
    "def store_user_history(data, score, feedback):\n",
    "    history = load_user_history()\n",
    "    entry = {\"vitals\": data, \"score\": score, \"feedback\": feedback}\n",
    "    history.append(entry)\n",
    "    with open(\"user_history.json\", \"w\") as file:\n",
    "        json.dump(history, file, indent=4)\n",
    "    if feedback:\n",
    "        with open(\"feedback.txt\", \"a\") as file:\n",
    "            file.write(f\"{feedback}\\n\")\n",
    "    embedding = embedding_model.encode(json.dumps(entry)).tolist()\n",
    "    index.add(np.array([embedding], dtype=np.float32))\n",
    "    faiss.write_index(index, \"user_history.index\")\n",
    "def retrieve_similar_cases(data):\n",
    "    if index.ntotal == 0:\n",
    "        return []\n",
    "    embedding = embedding_model.encode(json.dumps(data)).tolist()\n",
    "    distances, indices = index.search(np.array([embedding], dtype=np.float32), k=3)\n",
    "    history = load_user_history()\n",
    "    return [history[i] for i in indices[0] if i < len(history)]\n",
    "def predict_recovery_score(data):\n",
    "    prompt = f\"\"\"\n",
    "    Given vitals:\n",
    "    - Heart Rate: {data['heart_rate']}\n",
    "    - BP: {data['bp']}\n",
    "    - Oxygen Saturation: {data['oxygen_saturation']}\n",
    "    - Symptoms: {data['symptoms']}\n",
    "    Predict a recovery score between 0-100. Only give a score, no description.\n",
    "    \"\"\"\n",
    "    return llm.predict(prompt).strip()\n",
    "def generate_recovery_plan(score, feedback_history, user_history, similar_cases):\n",
    "    prompt = f\"\"\"\n",
    "    The patient's recovery score is {score}. Suggest a structured recovery plan.\n",
    "    Consider the following:\n",
    "    - Feedback History: {feedback_history}\n",
    "    - User History: {user_history}\n",
    "    - Similar Cases: {similar_cases}\n",
    "    Adjust the plan to address any concerns or requests raised in the feedback.\n",
    "    \"\"\"\n",
    "    return llm.predict(prompt).strip()\n",
    "def get_feedback_history():\n",
    "    if os.path.exists(\"feedback.txt\"):\n",
    "        with open(\"feedback.txt\", \"r\") as file:\n",
    "            return file.read()\n",
    "    return \"No feedback yet.\"\n",
    "def process_input(heart_rate, bp, oxygen_saturation, symptoms):\n",
    "    data = {\n",
    "        \"heart_rate\": heart_rate,\n",
    "        \"bp\": bp,\n",
    "        \"oxygen_saturation\": oxygen_saturation,\n",
    "        \"symptoms\": symptoms,\n",
    "    }\n",
    "    score = predict_recovery_score(data)\n",
    "    feedback_history = get_feedback_history()\n",
    "    user_history = load_user_history()\n",
    "    similar_cases = retrieve_similar_cases(data)\n",
    "    plan = generate_recovery_plan(score, feedback_history, user_history, similar_cases)\n",
    "    return score, plan, \"Please provide feedback on the generated recovery plan.\"\n",
    "def process_feedback(heart_rate, bp, oxygen_saturation, symptoms, feedback):\n",
    "    data = {\n",
    "        \"heart_rate\": heart_rate,\n",
    "        \"bp\": bp,\n",
    "        \"oxygen_saturation\": oxygen_saturation,\n",
    "        \"symptoms\": symptoms,\n",
    "    }\n",
    "    score = predict_recovery_score(data)\n",
    "    store_user_history(data, score, feedback)\n",
    "    feedback_history = get_feedback_history()\n",
    "    return f\"Thank you for your feedback: {feedback}. It has been stored successfully. Current Feedback History: {feedback_history}\"\n",
    "demo = gr.Blocks()\n",
    "with demo:\n",
    "    gr.Markdown(\"# AI Recovery Score & Plan Generator with Feedback\")\n",
    "    with gr.Row():\n",
    "        with gr.Column():\n",
    "            heart_rate = gr.Number(label=\"Heart Rate\")\n",
    "            bp = gr.Textbox(label=\"Blood Pressure\")\n",
    "            oxygen_saturation = gr.Textbox(label=\"Oxygen Saturation\")\n",
    "            symptoms = gr.Textbox(label=\"Symptoms\")\n",
    "            feedback = gr.Textbox(label=\"Feedback\", visible=False)\n",
    "            submit = gr.Button(\"Submit Vitals\")\n",
    "            feedback_button = gr.Button(\"Submit Feedback\", visible=False)\n",
    "        with gr.Column():\n",
    "            recovery_score = gr.Textbox(label=\"Predicted Recovery Score\", interactive=False)\n",
    "            recovery_plan = gr.Textbox(label=\"Recovery Plan\", interactive=False)\n",
    "            feedback_prompt = gr.Textbox(label=\"Feedback Prompt\", interactive=False)\n",
    "    submit.click(\n",
    "        process_input,\n",
    "        inputs=[heart_rate, bp, oxygen_saturation, symptoms],\n",
    "        outputs=[recovery_score, recovery_plan, feedback_prompt],\n",
    "    ).then(\n",
    "        lambda: (gr.update(visible=True), gr.update(visible=True)),\n",
    "        outputs=[feedback, feedback_button],\n",
    "    )\n",
    "    feedback_button.click(\n",
    "        process_feedback,\n",
    "        inputs=[heart_rate, bp, oxygen_saturation, symptoms, feedback],\n",
    "        outputs=[feedback_prompt],\n",
    "    )\n",
    "if __name__ == \"__main__\":\n",
    "    demo.launch(share=True)"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}